In [1]:
import os
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

from utils import process_image, get_classes, draw, convert_boxes

from YOLOv3.model.yolo_model import YOLO

from deep_sort.deep_sort import preprocessing
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.tools import generate_detections

/Users/connorsungczarnuch/projects/brainshare-tracking/venv/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [2]:
def detect_image(frame, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        frame: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        processed_frame: processed image.
    """
    processed_frame = process_image(frame)

    start = time.time()
    boxes, classes, scores = yolo.predict(processed_frame, frame.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(frame, boxes, scores, classes, all_classes)

    return frame, boxes, scores, classes

In [3]:
def tracking(video, yolo, all_classes):
    video_path = os.path.join("videos", "test", video) # Get absolute path of input video file
    camera = cv2.VideoCapture(video_path) # Open video path using OpenCV
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE) # Create OpenCV window for displaying output video

    # Prepare for saving the detected video
    size = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))) # Get size of input video
    fps = int(camera.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'h264') # Set codec for output video

    vout = cv2.VideoWriter() # Initialize video writer object
    vout.open(os.path.join("videos", "res", video), fourcc, fps, size, True) # Open video file to write to

    # Loop through an infinite while loop until the video stream ends
    while True:
        res, frame = camera.read() # Read next frame from video stream

        if not res:
            break # If there is no next frame, exit the loop

        frame, boxes, scores, classes = detect_image(frame, yolo, all_classes) # Perform detections using YOLOv3 to generate the bounding boxes, confidence scores, and class names of each detected object

        # Run conversions and create Detection objects
        features = encoder(frame, boxes)
        detections = [Detection(bbox, score, feature) for bbox, score, feature in zip(boxes, scores, features)]

        # Init colour map
        cmap = plt.get_cmap('tab20b')
        colours = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

        # Non-max suppression
        boxes = np.array([d.tlwh for d in detections]) # Generate a numpy array of detections using TLWH coordinates
        scores = np.array([d.confidence for d in detections]) # Generate a numpy array of confidence scores
        indices = preprocessing.non_max_suppression(boxes, classes, nms_max_overlap, scores) # Find indices of max-confidence level bounding box
        detections = [detections[i] for i in indices] # Filter detections based on max confidence level/non-max suppression

        tracker.predict() # Predict the next location of the bounding boxes using the deep_sort tracker
        tracker.update(detections) # Update the tracker with the actual locations of the objects

        # Check for confirmed tracks and draw on output stream
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1: # If track is not confirmed, do not draw on frame
                continue
            # If track is confirmed and "live", draw track on frame
            bbox = track.to_tlbr()
            colour = colours[int(track.track_id) % len(colours)]
            colour = [i * 255 for i in colour]
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), colour, 2)
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1]-30)), (int(bbox[0])+(len(str(track.track_id)))*17, int(bbox[1])), colour, -1)
            cv2.putText(frame, str(track.track_id),(int(bbox[0]), int(bbox[1]-10)),0, 0.75, (255,255,255),2)
        
        # Send updated frame to output and write to video file
        cv2.imshow("tracking", frame)
        vout.write(frame)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()
    cv2.destroyAllWindows()

In [4]:
yolo = YOLO(0.6, 0.5, 'YOLOv3/data/yolo.h5') # Initialize yolo model
file = 'YOLOv3/data/coco_classes.txt'
all_classes = get_classes(file)

# Tracker parameters:
    # max_cosine_distance: How far away can the current tracked box be from the previous? Scaled
    # nn_budget: limit the size of the neural network if on lower-end hardware
    # nms_max_overlap: When performing non-max suppression, how much overlap is allowed between two boxes
max_cosine_distance = 0.5
nn_budget = None
nms_max_overlap = 1.0

model_filename = 'deep_sort/resources/networks/mars-small128.pb'
encoder = generate_detections.create_box_encoder(model_filename, batch_size=1) # Encoder used to generate features for given bounding boxes
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget) # Metric to calculate which bounding box relates to which track
tracker = Tracker(metric)


# detect videos one at a time in videos/test folder    
video = 'library1.mp4'
tracking(video, yolo, all_classes)

8781004  464.28646088 196.19031044 420.35361803]
class: bicycle, score: 0.99
box coordinate x,y,w,h: [296.01869345 663.31504822 386.0112072  279.13948939]

time: 0.25s
class: person, score: 1.00
box coordinate x,y,w,h: [386.71128273 455.44654846 193.64541939 441.91072171]
class: bicycle, score: 0.99
box coordinate x,y,w,h: [292.92591333 661.69288635 392.66095464 280.91423695]

time: 0.26s
class: person, score: 1.00
box coordinate x,y,w,h: [390.13446808 472.00302124 180.80727972 413.70177269]
class: bicycle, score: 0.99
box coordinate x,y,w,h: [291.11988544 663.41064453 397.86920786 279.20557902]

time: 0.25s
class: person, score: 1.00
box coordinate x,y,w,h: [391.7975235  471.66564941 180.27435119 423.15338208]
class: person, score: 0.67
box coordinate x,y,w,h: [359.97208357 596.25274658 292.8040742  307.78627396]
class: bicycle, score: 0.99
box coordinate x,y,w,h: [286.43177032 658.40103149 414.2933244  288.2703561 ]

time: 0.25s
class: person, score: 0.98
box coordinate x,y,w,h: [414